# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_weather_data = pd.read_csv('city_weather_data.csv').drop(['Unnamed: 0'], axis=1) #drop extra index on csv import
city_weather_data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nikolskoye,75,RU,1612026470,93,59.7035,30.7861,-2.00,5.00
1,Korla,0,CN,1612026663,79,41.7597,86.1469,-3.89,2.65
2,San Nicolas,28,PH,1612026684,71,18.1725,120.5953,20.24,4.66
3,Hambantota,90,LK,1612026671,83,6.1241,81.1185,25.65,5.26
4,Chokurdakh,3,RU,1612026613,85,70.6333,147.9167,-33.85,3.18
...,...,...,...,...,...,...,...,...,...
725,Eskil,100,TR,1612038757,96,38.4017,33.4131,2.60,7.31
726,Hobyo,7,SO,1612038758,78,5.3505,48.5268,24.17,6.76
727,Lamar,90,US,1612038760,77,33.6668,-95.5836,20.00,7.72
728,Somerset East,15,ZA,1612038762,79,-32.7217,25.5880,16.71,4.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [15]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_data[["Lat", "Lng"]].astype(float)

# Store humidty as weight
humidity = city_weather_data["Humidity"].astype(float)

In [16]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#A max temperature lower than 35 degrees but higher than 30.
#Wind speed less than 10 mph.
#Less than 0% cloudiness

hotel_df = city_weather_data.loc[(city_weather_data["Max Temp"] > 30) & (city_weather_data["Max Temp"] < 35) & (city_weather_data["Wind Speed"] < 10) & (city_weather_data["Cloudiness"] == 0) ]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
10,Vallenar,0,CL,1612026698,18,-28.5708,-70.7581,31.71,5.22
96,Arraial do Cabo,0,BR,1612026850,48,-22.9661,-42.0278,34.00,8.75
121,Paita,0,PE,1612026893,45,-5.0892,-81.1144,32.00,5.66
255,Arraial do Cabo,0,BR,1612026850,48,-22.9661,-42.0278,34.00,8.75
386,Arraial do Cabo,0,BR,1612026850,48,-22.9661,-42.0278,34.00,8.75
503,Arraial do Cabo,0,BR,1612027564,48,-22.9661,-42.0278,34.00,8.75
515,Tomatlán,0,MX,1612027586,25,19.9333,-105.2500,33.10,1.91
660,Arraial do Cabo,0,BR,1612027564,48,-22.9661,-42.0278,34.00,8.75
693,Arraial do Cabo,0,BR,1612027564,48,-22.9661,-42.0278,34.00,8.75
703,Manga,0,BF,1612027915,13,11.6636,-1.0731,30.25,2.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.dropna().drop_duplicates("City") #remove duplicates.
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
10,Vallenar,0,CL,1612026698,18,-28.5708,-70.7581,31.71,5.22,
96,Arraial do Cabo,0,BR,1612026850,48,-22.9661,-42.0278,34.00,8.75,
121,Paita,0,PE,1612026893,45,-5.0892,-81.1144,32.00,5.66,
515,Tomatlán,0,MX,1612027586,25,19.9333,-105.2500,33.10,1.91,
703,Manga,0,BF,1612027915,13,11.6636,-1.0731,30.25,2.80,


In [24]:
# Fixed params independent of each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use the lat/lng 
for index, row in hotel_df.iterrows():
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    if row["Hotel Name"]=="":

        # change location params
        params["location"] = f"{lat},{lng}"

        # make request 
        name = requests.get(base_url, params=params).json()
        

        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]

In [25]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
10,Vallenar,0,CL,1612026698,18,-28.5708,-70.7581,31.71,5.22,Humacao Bed & Breakfast
96,Arraial do Cabo,0,BR,1612026850,48,-22.9661,-42.0278,34.00,8.75,Pousada Porto Praia
121,Paita,0,PE,1612026893,45,-5.0892,-81.1144,32.00,5.66,Hotel Náutico de Paita
515,Tomatlán,0,MX,1612027586,25,19.9333,-105.2500,33.10,1.91,Norma franco
703,Manga,0,BF,1612027915,13,11.6636,-1.0731,30.25,2.80,Auberge du 11 décembre de la cité CNSS à Manga


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,info_box_content= hotel_info)
fig = gmaps.figure(center=(0,0), zoom_level=2.0)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))